In [22]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
import rasterio 
import matplotlib.pyplot as plt
from rasterio.plot import show

# Load input data
## Administrative boundaries

In [13]:
# load administrative boundaries
boundaries_cities = gpd.read_file('https://storage.googleapis.com/data_portal_exposure/data/administrative_boundaries/mapped/boundary_CHL_vitacura_mapped_geom.geojson')
# add centroid coordinates
boundaries_cities["centroid_x"] = boundaries_cities.centroid.x
boundaries_cities["centroid_y"] = boundaries_cities.centroid.y

C:\Users\SAIF~1.SHA\AppData\Local\Temp/ipykernel_14160/570289728.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  boundaries_cities["centroid_x"] = boundaries_cities.centroid.x
C:\Users\SAIF~1.SHA\AppData\Local\Temp/ipykernel_14160/570289728.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  boundaries_cities["centroid_y"] = boundaries_cities.centroid.y


In [27]:
boundaries_cities.head()

,country_iso3,city_id,city_name,boundary_data_source,geometry,centroid_x,centroid_y
0,CHL,CHL-Vitacura,Vitacura,osm,"MULTIPOLYGON (((-70.61347 -33.40884, -70.61207...",-70.572532,-33.37915


## Amenities

In [2]:
# get amenity data
amenity_dataportal_cities = gpd.read_file('https://storage.googleapis.com/data_portal_exposure/data/amenity/amenity_Vitacura.geojson')
amenity_dataportal_cities.head()

,featureCategory,featureType,id,latitude,longitude,cityName,integrationDate,objectType,projectName,geometry
0,amenity,pharmacy,240434655,-33.376238,-70.525053,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.52505 -33.37624)
1,amenity,parking,253242064,-33.393896,-70.600037,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.60004 -33.39390)
2,amenity,parking,253272975,-33.402910,-70.580557,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.58056 -33.40291)
3,amenity,parking,253276941,-33.392850,-70.575862,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.57586 -33.39285)
4,amenity,parking,253433629,-33.403386,-70.579463,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.57946 -33.40339)


In [5]:
# get amenity sectors
amenity_sectors=pd.read_csv("https://storage.googleapis.com/data_portal_exposure/data/amenity/param/osm_amenity_sectors_gcom.csv")
amenity_sectors.head()

,sector_name,gcom_sector_name,feature_key,feature_value,feature_category
0,Commercial,Commercial,building,\tretail,Commercial
1,Commercial,Commercial,building,warehouse,Commercial
2,Commercial,Commercial,building,commercial,Commercial
3,Commercial,Commercial,building,industrial,Commercial
4,Commercial,Commercial,amenity,marketplace,Others


In [10]:
# join amenity loaction with sector names
amenity_dataportal_cities = amenity_dataportal_cities.merge(amenity_sectors, 
                                     left_on='featureType',
                                     right_on='feature_value',
                                     how = "left")

In [11]:
# replace nan
amenity_dataportal_cities['sector_name'] = amenity_dataportal_cities['sector_name'].replace(np.nan, 'Other')
amenity_dataportal_cities['gcom_sector_name'] = amenity_dataportal_cities['gcom_sector_name'].replace(np.nan, 'Other')

In [26]:
amenity_dataportal_cities.head()

,featureCategory,featureType,id,latitude,longitude,cityName,integrationDate,objectType,projectName,geometry,sector_name,gcom_sector_name,feature_key,feature_value,feature_category
0,amenity,pharmacy,240434655,-33.376238,-70.525053,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.52505 -33.37624),Healthcare,Public Health,amenity,pharmacy,Healthcare
1,amenity,parking,253242064,-33.393896,-70.600037,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.60004 -33.39390),Transportation,Transport,amenity,parking,Transportation
2,amenity,parking,253272975,-33.402910,-70.580557,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.58056 -33.40291),Transportation,Transport,amenity,parking,Transportation
3,amenity,parking,253276941,-33.392850,-70.575862,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.57586 -33.39285),Transportation,Transport,amenity,parking,Transportation
4,amenity,parking,253433629,-33.403386,-70.579463,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.57946 -33.40339),Transportation,Transport,amenity,parking,Transportation


## Land Surface temperature

In [18]:
# Load LST

# seelect city id
city_id = "CHL-Vitacura"
# define path
fpath = "https://storage.googleapis.com/data_portal_exposure/data/land_surface_temperature/lst_mean/lst_"+city_id+".tif"
# open raster data
city_lst_mean_r = rasterio.open(fpath)
# print infos
print(city_lst_mean_r.profile)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 356, 'height': 219, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, -70.61351021266958,
       0.0, -0.00026949458523585647, -33.35049391210771), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}


In [19]:
# read raster data for the selected city
city_lst_mean = city_lst_mean_r.read(1)
print(city_lst_mean)
print(city_lst_mean.mean())

[[30.57653359 30.57653359 29.77059886 ... 30.04558745 29.91679579
  29.77179447]
 [30.89397511 30.89397511 30.07795435 ... 30.68595631 30.51210315
  30.42295177]
 [31.36221366 31.36221366 30.68852586 ... 31.34067878 31.44456609
  31.32332093]
 ...
 [30.82699103 31.23492598 31.82480174 ... 30.82131581 30.71491055
  30.71491055]
 [30.27306214 30.84194096 31.88186035 ... 30.89071381 30.80060106
  30.80060106]
 [29.83294584 30.53301595 31.80662613 ... 30.77605415 30.70221404
  30.70221404]]
32.50379025664531


# Compute amenity exposure

In [34]:
amenity_dataportal_cities.head()

,featureCategory,featureType,id,latitude,longitude,cityName,integrationDate,objectType,projectName,geometry,sector_name,gcom_sector_name,feature_key,feature_value,feature_category,country_iso3,city_id,city_name
0,amenity,pharmacy,240434655,-33.376238,-70.525053,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.52505 -33.37624),Healthcare,Public Health,amenity,pharmacy,Healthcare,CHL,CHL-Vitacura,Vitacura
1,amenity,parking,253242064,-33.393896,-70.600037,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.60004 -33.39390),Transportation,Transport,amenity,parking,Transportation,CHL,CHL-Vitacura,Vitacura
2,amenity,parking,253272975,-33.402910,-70.580557,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.58056 -33.40291),Transportation,Transport,amenity,parking,Transportation,CHL,CHL-Vitacura,Vitacura
3,amenity,parking,253276941,-33.392850,-70.575862,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.57586 -33.39285),Transportation,Transport,amenity,parking,Transportation,CHL,CHL-Vitacura,Vitacura
4,amenity,parking,253433629,-33.403386,-70.579463,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.57946 -33.40339),Transportation,Transport,amenity,parking,Transportation,CHL,CHL-Vitacura,Vitacura


In [33]:
# join amenity loaction with administrative boundaries data to get city_id filed
amenity_dataportal_cities = amenity_dataportal_cities.merge(boundaries_cities[["country_iso3", "city_id","city_name"]], 
                                     left_on='cityName',
                                     right_on='city_name',
                                     how = "left")

In [45]:
# create empty dataframe
amenity_exposure = pd.DataFrame(columns=['city_name',
                                         'city_lst_avg',
                                         'city_lst_perc_90',
                                         'feature_key',
                                         'feature_value',
                                         'feature_category',
                                         'latitude',
                                         'longitude',
                                         'geometry',
                                         'country_iso',
                                         'city_id',
                                         'sector_name',
                                         'gcom_sector_name',
                                         'exposure_lst_mean',
                                         'lst_value_dev_city_mean',
                                         'lst_pecent_dev_city_mean',
                                         'lst_dev_city_mean_exposure_class',
                                         'lst_value_dev_city_perc_90',
                                         'lst_pecent_dev_city_perc_90',
                                         'lst_dev_city_perc90_exposure_class'
                                         ])

In [46]:

for i in range(len(boundaries_cities)):
    city_id = boundaries_cities.loc[i, 'city_id']
    print("\n City Id: "+city_id)
    # select amenity data for the selectd city
    city_amenity =  amenity_dataportal_cities[amenity_dataportal_cities.city_id == city_id]
    # select LST data
    fpath = "https://storage.googleapis.com/data_portal_exposure/data/land_surface_temperature/lst_mean/lst_"+city_id+".tif"
    print("\n path: "+fpath)
    city_lst_mean_r = rasterio.open(fpath)
    print(city_lst_mean_r)
    # get amenity coordinates
    city_amenity_coords = [(x,y) for x, y in zip(city_amenity.longitude, city_amenity.latitude)]
    # extract lst values for amenity coordinates
    city_amenity['exposure_lst_mean'] = [x[0] for x in city_lst_mean_r.sample(city_amenity_coords)]
    # get average lst value within all city raster area
    city_lst_mean_r_band1= city_lst_mean_r.read(1)
    city_lst_mean_r_mean = np.nanmean(city_lst_mean_r_band1)
    city_amenity['city_lst_avg'] = city_lst_mean_r_mean
    # get 90th percentile value within all city raster area
    city_lst_mean_r_perc_90 = np.nanpercentile(city_lst_mean_r_band1, 90)
    city_amenity['city_lst_perc_90'] = city_lst_mean_r_perc_90
    # compute mean-based indicators
    city_amenity['lst_value_dev_city_mean'] = round(city_amenity['exposure_lst_mean'] - city_amenity['city_lst_avg'],2)
    city_amenity['lst_pecent_dev_city_mean'] = round((city_amenity['exposure_lst_mean'] * 100)/city_amenity['city_lst_avg'],2)
    city_amenity.loc[city_amenity['lst_value_dev_city_mean'] > 0, 'lst_dev_city_mean_exposure_class'] = 'Exposed'
    city_amenity.loc[city_amenity['lst_value_dev_city_mean'] < 0, 'lst_dev_city_mean_exposure_class'] = 'Not Exposed'
    # compute 90 percentile-based indicators
    city_amenity['lst_value_dev_city_perc_90'] = round(city_amenity['exposure_lst_mean'] - city_amenity['city_lst_perc_90'],2)
    city_amenity['lst_pecent_dev_city_perc_90'] = round((city_amenity['exposure_lst_mean'] * 100)/city_amenity['city_lst_perc_90'],2)
    city_amenity.loc[city_amenity['lst_value_dev_city_perc_90'] > 0, 'lst_dev_city_perc90_exposure_class'] = 'Exposed'
    city_amenity.loc[city_amenity['lst_value_dev_city_perc_90'] < 0, 'lst_dev_city_perc90_exposure_class'] = 'Not Exposed'
    # append with intiailized dataframe
    amenity_exposure = amenity_exposure.append(city_amenity, ignore_index=True)
    


 City Id: CHL-Vitacura

 path: https://storage.googleapis.com/data_portal_exposure/data/land_surface_temperature/lst_mean/lst_CHL-Vitacura.tif
<open DatasetReader name='https://storage.googleapis.com/data_portal_exposure/data/land_surface_temperature/lst_mean/lst_CHL-Vitacura.tif' mode='r'>


In [48]:
# convert into csv
amenity_exposure_csv = amenity_exposure[['city_name',
                                         'city_lst_avg',
                                         'city_lst_perc_90',
                                         'feature_key',
                                         'feature_value',
                                         'feature_category',
                                         'latitude',
                                         'longitude',
                                         'geometry',
                                         'country_iso',
                                         'city_id',
                                         'sector_name',
                                         'gcom_sector_name',
                                         'exposure_lst_mean',
                                         'lst_value_dev_city_mean',
                                         'lst_pecent_dev_city_mean',
                                         'lst_dev_city_mean_exposure_class',
                                         'lst_value_dev_city_perc_90',
                                         'lst_pecent_dev_city_perc_90',
                                         'lst_dev_city_perc90_exposure_class'
                                         ]].to_csv()

In [51]:
amenity_exposure.head()

,city_name,city_lst_avg,city_lst_perc_90,feature_key,feature_value,feature_category,latitude,longitude,geometry,country_iso,...,lst_pecent_dev_city_perc_90,lst_dev_city_perc90_exposure_class,featureCategory,featureType,id,cityName,integrationDate,objectType,projectName,country_iso3
0,Vitacura,32.50379,35.864247,amenity,pharmacy,Healthcare,-33.376238,-70.525053,POINT (-70.52505 -33.37624),NaN,...,90.79,Not Exposed,amenity,pharmacy,240434655.0,Vitacura,2021-09-23,amenity,dataportal,CHL
1,Vitacura,32.50379,35.864247,amenity,parking,Transportation,-33.393896,-70.600037,POINT (-70.60004 -33.39390),NaN,...,90.54,Not Exposed,amenity,parking,253242064.0,Vitacura,2021-09-23,amenity,dataportal,CHL
2,Vitacura,32.50379,35.864247,amenity,parking,Transportation,-33.402910,-70.580557,POINT (-70.58056 -33.40291),NaN,...,87.83,Not Exposed,amenity,parking,253272975.0,Vitacura,2021-09-23,amenity,dataportal,CHL
3,Vitacura,32.50379,35.864247,amenity,parking,Transportation,-33.392850,-70.575862,POINT (-70.57586 -33.39285),NaN,...,92.89,Not Exposed,amenity,parking,253276941.0,Vitacura,2021-09-23,amenity,dataportal,CHL
4,Vitacura,32.50379,35.864247,amenity,parking,Transportation,-33.403386,-70.579463,POINT (-70.57946 -33.40339),NaN,...,87.44,Not Exposed,amenity,parking,253433629.0,Vitacura,2021-09-23,amenity,dataportal,CHL


In [49]:
amenity_exposure.to_csv('amenity_exposure_heat.csv')

# Plot

In [16]:
#######################################################################################################################
#  plot amenity within one city  
#######################################################################################################################

# select city name
city_name = "Vitacura"
# select amenity data for the selectd city
city_amenity =  amenity_dataportal_cities[amenity_dataportal_cities.cityName == city_name]
# select boundaries data for the selected city
city_boundaries =  boundaries_cities[boundaries_cities.city_name == city_name]

# color definition in boundary polygon
style_function_Boundaries = lambda x: {
    'fillColor':"gray",
    'color': "black",
    'weight': 3.5,
    'fillOpacity': 0.3
}


# Plot map background
m = folium.Map(location=[float(city_boundaries["centroid_y"]), float(city_boundaries["centroid_x"])],
               zoom_start = 8)


# plot administrative boundaries of selected city
folium.GeoJson(
    city_boundaries,
    name = "Administrative boundaries",
    style_function=style_function_Boundaries,
).add_to(m)

# plot amenities of the selected city
for i in range(0,len(city_amenity)):
    folium.CircleMarker(
        location=[city_amenity.iloc[i]['latitude'], city_amenity.iloc[i]['longitude']],
        popup=city_amenity.iloc[i]['featureType'],
        radius=2,
        color="red",
        fill_color="red",
        fill=True,
        name = "amenities",
    ).add_to(m)

folium.LayerControl().add_to(m)

m